In [ ]:
!pip install flask flask-ngrok flask-cors transformers tensorflow pyngrok


In [ ]:
!ngrok authtoken 2spbWZszlYSTSI8KbUfE16df8XI_2ddKZWvBBquWLvKhWjU6E


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile

# Paths to zip files in Google Drive
suicide_model_zip = "/content/drive/MyDrive/bert_suicide_model.zip"
emotion_model_zip = "/content/drive/MyDrive/bert_emotion_model.zip"
tokenizer_zip = "/content/drive/MyDrive/tokenizer (1).zip"

# Unzip models and tokenizer
unzip_paths = {
    suicide_model_zip: "/content/bert_suicide_model",
    emotion_model_zip: "/content/bert_emotion_model",
    tokenizer_zip: "/content/tokenizer (1)"
}

for zip_path, extract_path in unzip_paths.items():
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

print("Models and tokenizer extracted successfully!")


Mounted at /content/drive
Models and tokenizer extracted successfully!


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer

# Load models
suicide_model = tf.saved_model.load("/content/bert_suicide_model")
emotion_model = tf.saved_model.load("/content/bert_emotion_model")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer (1)")

print("Models and tokenizer loaded successfully!")


Models and tokenizer loaded successfully!


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import tensorflow as tf
import pandas as pd
import csv
import os

app = Flask(__name__)
CORS(app)  # Allow cross-origin requests (Fixes CORS issue)

# Expose the app via ngrok
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

# Suicide and emotion detection function
def detect_suicide_and_emotion(text):
    suicide_label_mapping = {0: "non-suicide", 1: "suicide"}
    emotion_label_mapping = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
    risk_levels = {"sadness": "Medium Risk", "anger": "Medium Risk", "fear": "Medium Risk"}

    # Tokenize and predict suicide risk
    try:
        inputs_suicide = tokenizer([text], padding="max_length", truncation=True, max_length=128, return_tensors="tf")
        inputs_suicide = {key: tf.cast(value, tf.int64) for key, value in inputs_suicide.items()}
        infer_suicide = suicide_model.signatures['serving_default']
        suicide_outputs = infer_suicide(**inputs_suicide)
        suicide_prediction = tf.argmax(suicide_outputs['output_1'], axis=1).numpy()[0]
        suicide_label = suicide_label_mapping[suicide_prediction]
    except Exception as e:
        return {"error": f"Error in suicide detection: {str(e)}"}

    if suicide_label == "suicide":
        result = {"text": text, "suicide_risk": "High Risk", "emotion": "Not Analyzed"}
        return result

    # Tokenize and predict emotion
    try:
        inputs_emotion = tokenizer([text], padding="max_length", truncation=True, max_length=87, return_tensors="tf")
        inputs_emotion = {key: tf.cast(value, tf.int64) for key, value in inputs_emotion.items()}
        infer_emotion = emotion_model.signatures['serving_default']
        emotion_outputs = infer_emotion(**inputs_emotion)
        emotion_prediction = tf.argmax(emotion_outputs['output_1'], axis=1).numpy()[0]
        emotion_label = emotion_label_mapping[emotion_prediction]
    except Exception as e:
        return {"error": f"Error in emotion detection: {str(e)}"}

    risk_level = risk_levels.get(emotion_label, "Low Risk")
    result = {"text": text, "emotion": emotion_label, "suicide_risk": risk_level}
    return result

# Flask API Route
@app.route("/analyze", methods=["POST"])
def analyze():
    try:
        data = request.get_json()
        text = data.get("text", "").strip()

        if not text:
            return jsonify({"error": "Text input is required!"}), 400

        result = detect_suicide_and_emotion(text)
        return jsonify(result)

    except Exception as e:
        return jsonify({"error": f"Internal Server Error: {str(e)}"}), 500

if __name__ == "__main__":
    app.run(port=5000)



Public URL: https://d4c8-34-124-237-232.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [24/Feb/2025 05:14:33] "OPTIONS /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Feb/2025 05:14:37] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Feb/2025 05:33:15] "OPTIONS /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Feb/2025 05:33:16] "POST /analyze HTTP/1.1" 200 -
